<a href="https://colab.research.google.com/github/vijayrgopu/apache_beam/blob/master/8_Side_Inputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install --quiet apache_beam

     |████████████████████████████████| 3.4MB 2.8MB/s 
     |████████████████████████████████| 1.2MB 43.4MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 59.2MB 61kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 225kB 56.2MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 112kB 59.8MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: google-api-python-client 1.7.12 has requirement httplib2<1dev,>=0.17.0, but you'll have httplib2 0.12.0 which is incompatible.
ERROR: chainer 6.5.0 has requirement typing<=3.6.6, but you'll have typing 3.7.4.1 which is incompatible.
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.1 which is incompatible.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving dept_data.txt to dept_data.txt


In [0]:
!touch exclude_ids.txt
!echo 149633CM >> exclude_ids.txt
!echo 212539MU >> exclude_ids.txt
!echo 231555ZZ >> exclude_ids.txt
!echo 704275DC >> exclude_ids.txt

ERROR! Session/line number was not unique in database. History logging moved to new session 65
total 48
drwxr-xr-x 1 root root  4096 Mar 18 16:23 sample_data
-rw-r--r-- 1 root root 33086 Apr  1 04:27 dept_data.txt
-rw-r--r-- 1 root root    36 Apr  1 04:29 exclude_ids.txt
drwxr-xr-x 2 root root  4096 Apr  1 05:50 beam-temp-output.txt-a82789c473dc11eaab070242ac1c0002
-rw-r--r-- 1 root root     0 Apr  1 05:54 output.txt-00000-of-00001


In [0]:
import apache_beam as beam

In [0]:
class FilterRecords(beam.DoFn):
  def process(self,element,side_input,low_bound,up_bound): #element will take a PCollection
    emp_id = element.split(',')[0]
    emp_name = element.split(',')[1]
    element_list = element.split(',')
    if (low_bound <= len(emp_name) <= up_bound) and emp_id not in side_input:
      return [element_list]

exclusion_list = list()
with open('exclude_ids.txt','r') as si_file:
  for line in si_file:
    exclusion_list.append(line.rstrip()) 

with beam.Pipeline() as p1:
  
  filtered_data = (
      p1
      |'Read file' >> beam.io.ReadFromText('dept_data.txt')
 #    |'Create a Map' >> beam.Map(lambda lines : lines.split(','))
      |'Call Filter Records' >> beam.ParDo(FilterRecords(),exclusion_list,3,10)
      |'Write To File' >> beam.io.WriteToText('output.txt')
  )


In [0]:
!head -20 output.txt-00000-of-00001

['149633CM', 'Marco', '10', 'Accounts', '1-01-2019']
['212539MU', 'Rebekah', '10', 'Accounts', '1-01-2019']
['231555ZZ', 'Itoe', '10', 'Accounts', '1-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '1-01-2019']
['704275DC', 'Kyle', '10', 'Accounts', '1-01-2019']
['957149WC', 'Kyle', '10', 'Accounts', '1-01-2019']
['241316NX', 'Kumiko', '10', 'Accounts', '1-01-2019']
['796656IE', 'Gaston', '10', 'Accounts', '1-01-2019']
['331593PS', 'Beryl', '20', 'HR', '1-01-2019']
['560447WH', 'Olga', '20', 'HR', '1-01-2019']
['222997TJ', 'Leslie', '20', 'HR', '1-01-2019']
['171752SY', 'Mindy', '20', 'HR', '1-01-2019']
['153636AS', 'Vicky', '20', 'HR', '1-01-2019']
['745411HT', 'Richard', '20', 'HR', '1-01-2019']
['298464HN', 'Kirk', '20', 'HR', '1-01-2019']
['783950BW', 'Kaori', '20', 'HR', '1-01-2019']
['892691AR', 'Beryl', '20', 'HR', '1-01-2019']
['245668UZ', 'Oscar', '20', 'HR', '1-01-2019']
['231206QD', 'Kumiko', '30', 'Finance', '1-01-2019']
['357919KT', 'Wendy', '30', 'Finance', '1-01-2019

In [0]:
!cat exclude_ids.txt
!ls -ltr
#!grep -e '503996WI' output.txt-00000-of-00001

149633CM
212539MU
231555ZZ
704275DC
total 88
drwxr-xr-x 1 root root  4096 Mar 18 16:23 sample_data
-rw-r--r-- 1 root root 33086 Apr  1 04:27 dept_data.txt
-rw-r--r-- 1 root root    36 Apr  1 04:29 exclude_ids.txt
drwxr-xr-x 2 root root  4096 Apr  1 05:50 beam-temp-output.txt-a82789c473dc11eaab070242ac1c0002
-rw-r--r-- 1 root root 39898 Apr  1 06:14 output.txt-00000-of-00001
['503996WI', 'Edouard', '10', 'Accounts', '1-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '2-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '3-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '4-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '5-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '6-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '7-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '8-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '9-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '10-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '11-01-2019']
['503996WI', 'Ed


149633CM
212539MU
231555ZZ
704275DC